In [4]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_wallet.ipynb

## Block Header

The `Header` is a summary of the most important facts about a block. It contains the following:
- `root`: Root of the Merkle Tree of TXs
- `prev_hash`: Hash of the previous block
- `number`: Block number
- `n_txs`: Number of txs included in the block
- `mined`: Does the header belong to a block that is mined
- `time`: Time of creation

The actual mining happens on the block header.

In [5]:
class Header(Hashable):
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.ctime()

In [6]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          💳 0x4d94eb5b9b...3d3
hash:          🔞 0xb8b11b9251...5c4
prev_hash:     🔸 0xd2663b66a8...f00
number:        2
n_txs:         10
mined:         False
time:          Sun Apr 11 20:51:50 2021


If anything in the block header changes its `hash` changes automatically as well.

In [8]:
bh_changed        = deepcopy(bh)
bh_changed.number = 120
assert bh != bh_changed

### Info

Includes all extra informations that are irrelevant for mining and therefore are not in the block header. These are the total txs `volume` and `fees` included in a block.

In [9]:
class Info(Hashable):
    def __init__(self, txs): 
        self.volume = sum([tx.value for tx in txs])
        self.fees   = sum([tx.fee   for tx in txs])

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [10]:
class Block(Hashable): 
    def __init__(self, bh, txs):
        self.info   = Info(txs)
        self.header = bh
        self.mt     = MerkleTree(txs)
        self.txs    = self.val_txs(txs)
        self.hash   = bh.hash
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),       'tx signature invalid'
        assert self.are_unique(txs),             'txs include duplicates'
        assert self.mt.root == self.header.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def json(self): 
        info = self.info.__dict__
        h    = self.header.__dict__
        txs  = [tx.json() for tx in self.txs.values()]
        d    = {**info, **h, 'txs': txs}
        return json.dumps(d, indent=4)
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __getitem__(self, key):return self.txs[key] 
    def __str__(self):         return (str(self.header)
                                       +'\n'+str(self.info)
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(tx.smry() for tx in self.txs.values()))

Create random list of signed txs.

In [14]:
w1,w2 = Wallet(),Wallet()
def r_stxs(n): return [w1.signed_tx(w2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [15]:
txs = r_stxs(4)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          🔶 0xd0d28a0c53...53f
hash:          👻 0x1520fd5335...2fb
prev_hash:     🔌 0xa622851359...2d2
number:        2
n_txs:         4
mined:         False
time:          Sun Apr 11 20:53:49 2021
volume:        28.0 eth
hash:          📤 0x7e39fb4082...477
fees:          0.21 eth

txs:
📮 0x88a6 -> 📽 0x97a7 7.0 eth
📮 0x88a6 -> 📽 0x97a7 9.0 eth
📮 0x88a6 -> 📽 0x97a7 8.0 eth
📮 0x88a6 -> 📽 0x97a7 4.0 eth


### Block JSON

Every block has a json representation that we will use for our API later.

In [16]:
block_json = b.json()

We can load this JSON string `d` and create a block object again.

In [17]:
def load_block(d):
    d  = json.loads(d)
    txs = []
    for tx in d['txs']: txs.append(load_tx(tx))
    bh = Header(d['root'],d['prev_hash'],d['number'],d['n_txs']) 
    for k,v in d.items(): setattr(bh, k, v)
    bh.hash = d['hash']
    b = Block(bh, txs)
    return b
    
b_from_json = load_block(b.json())

In [18]:
assert b == b_from_json

#### Access specific tx from block with its hash.

In [19]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0x28de3ee962255c13968ef8497966194f272085d4ea2c91a35239c13115608f92'

In [20]:
tx = b[tx_hash]; print(tx)

fr:            📮 0x88a629B7F4...b55
hash:          💎 0x28de3ee962...f92
to:            📽 0x97a78D775b...4a1
value:         7.0 eth
fee:           0.08 eth
nonce:         0
time:          Sun Apr 11 20:53:49 2021
signed:        True
sig:           🕒 0xeca9040241...f1c


Blocks can only contain unique txs. 

In [21]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [22]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected